In [1]:
import pandas as pd
from sklearn.utils import resample
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from sklearn.impute import SimpleImputer
import joblib


In [2]:
#load dataset
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

In [3]:
#rebalance dataset
class_0 = df[df['TARGET'] == 0]
class_1 = df[df['TARGET'] == 1]

# balance mayority class with minority
sub_class = resample(class_0, replace=False, n_samples=len(class_1), random_state=42)

# combine
df = pd.concat([sub_class, class_1])

In [4]:
y = df['TARGET']
X = df.drop(columns=['TARGET', 'SK_ID_CURR'])  #adde sk_id_curr

In [5]:
#manages divided by 0 values
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [6]:
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="constant", fill_value= 0)  #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [7]:
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y ,random_state=42)

In [9]:
catboost_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'PRAUC',
    'custom_metric': ['AUC', 'PRAUC'],

    'auto_class_weights': 'Balanced',

    # Boosting
    'iterations': 4000,
    'learning_rate': 0.025,
    'depth': 7,

    # Regularización
    'l2_leaf_reg': 5.0,
    'random_strength': 1.5,

    # Bootstrap
    'bootstrap_type': 'Bayesian',
    'bagging_temperature': 0.7,

    # Growing
    'grow_policy': 'SymmetricTree',
    'min_data_in_leaf': 15,

    # Categóricas
    'one_hot_max_size': 12,

    # GPU
    'task_type': 'GPU',
    'devices': '0',

    # Control
    'random_seed': 42,
    'early_stopping_rounds': 250,
    'verbose': 200,}


In [10]:
#catboost features and kfold
cat_features = [
    col for col in X.columns
    if X[col].dtype == 'category'
]

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
aucs = []
models = [] #save models for later

for fold, (tr, va) in enumerate(skf.split(X_train, y_train), 1):
    model = CatBoostClassifier(**catboost_params)

    model.fit(
        X_train.iloc[tr], y_train.iloc[tr],
        eval_set=(X_train.iloc[va], y_train.iloc[va]),
        cat_features=cat_features)

    preds = model.predict_proba(X_train.iloc[va])[:, 1]
    auc = roc_auc_score(y_train.iloc[va], preds)
    aucs.append(auc)
    models.append(model)

    print(f'Fold {fold} AUC: {auc:.5f}')

print(f'\nAUC medio: {np.mean(aucs):.5f} ± {np.std(aucs):.5f}')



Default metric period is 5 because AUC, PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7199583	test: 0.7099537	best: 0.7099537 (0)	total: 192ms	remaining: 12m 49s
200:	learn: 0.7882336	test: 0.7573557	best: 0.7573557 (200)	total: 17.7s	remaining: 5m 34s
400:	learn: 0.8114244	test: 0.7647214	best: 0.7647214 (400)	total: 36.3s	remaining: 5m 25s
600:	learn: 0.8299175	test: 0.7674401	best: 0.7674628 (597)	total: 53.8s	remaining: 5m 4s
800:	learn: 0.8431048	test: 0.7681089	best: 0.7681089 (800)	total: 1m 10s	remaining: 4m 42s
1000:	learn: 0.8544570	test: 0.7682677	best: 0.7683192 (996)	total: 1m 27s	remaining: 4m 22s
1200:	learn: 0.8639915	test: 0.7686769	best: 0.7688249 (1172)	total: 1m 44s	remaining: 4m 4s
1400:	learn: 0.8701980	test: 0.7685790	best: 0.7688249 (1172)	total: 2m 1s	remaining: 3m 44s
bestTest = 0.7688248677
bestIteration = 1172
Shrink model to first 1173 iterations.
Fold 1 AUC: 0.77696


Default metric period is 5 because AUC, PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7231086	test: 0.7207235	best: 0.7207235 (0)	total: 94ms	remaining: 6m 15s
200:	learn: 0.7872727	test: 0.7646213	best: 0.7646213 (200)	total: 22.1s	remaining: 6m 57s
400:	learn: 0.8098452	test: 0.7706381	best: 0.7706381 (400)	total: 39.6s	remaining: 5m 55s
600:	learn: 0.8279209	test: 0.7727384	best: 0.7727715 (598)	total: 56.7s	remaining: 5m 20s
800:	learn: 0.8423905	test: 0.7738167	best: 0.7738167 (800)	total: 1m 13s	remaining: 4m 54s
1000:	learn: 0.8535287	test: 0.7742814	best: 0.7744030 (981)	total: 1m 30s	remaining: 4m 31s
1200:	learn: 0.8643875	test: 0.7737903	best: 0.7744030 (981)	total: 1m 48s	remaining: 4m 12s
bestTest = 0.7744029735
bestIteration = 981
Shrink model to first 982 iterations.
Fold 2 AUC: 0.78093


Default metric period is 5 because AUC, PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7234884	test: 0.7143393	best: 0.7143393 (0)	total: 95ms	remaining: 6m 20s
200:	learn: 0.7902147	test: 0.7490423	best: 0.7491274 (198)	total: 19.2s	remaining: 6m 3s
400:	learn: 0.8121602	test: 0.7549246	best: 0.7549346 (399)	total: 37.9s	remaining: 5m 40s
600:	learn: 0.8289939	test: 0.7565416	best: 0.7570476 (579)	total: 54.8s	remaining: 5m 9s
800:	learn: 0.8449308	test: 0.7580641	best: 0.7581187 (794)	total: 1m 11s	remaining: 4m 47s
1000:	learn: 0.8584771	test: 0.7585277	best: 0.7585429 (998)	total: 1m 29s	remaining: 4m 26s
1200:	learn: 0.8681182	test: 0.7585005	best: 0.7586471 (1169)	total: 1m 45s	remaining: 4m 6s
1400:	learn: 0.8779053	test: 0.7589648	best: 0.7589857 (1374)	total: 2m 2s	remaining: 3m 47s
1600:	learn: 0.8857344	test: 0.7585050	best: 0.7590765 (1422)	total: 2m 20s	remaining: 3m 30s
bestTest = 0.7590764693
bestIteration = 1422
Shrink model to first 1423 iterations.
Fold 3 AUC: 0.77720


Default metric period is 5 because AUC, PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7214450	test: 0.7219691	best: 0.7219691 (0)	total: 97.9ms	remaining: 6m 31s
200:	learn: 0.7890041	test: 0.7635444	best: 0.7635444 (200)	total: 19.4s	remaining: 6m 6s
400:	learn: 0.8108972	test: 0.7690438	best: 0.7690438 (400)	total: 36.9s	remaining: 5m 31s
600:	learn: 0.8287099	test: 0.7709352	best: 0.7710901 (590)	total: 54.3s	remaining: 5m 7s
800:	learn: 0.8447725	test: 0.7714947	best: 0.7717072 (744)	total: 1m 11s	remaining: 4m 46s
1000:	learn: 0.8575785	test: 0.7720298	best: 0.7720298 (1000)	total: 1m 29s	remaining: 4m 27s
1200:	learn: 0.8673329	test: 0.7718103	best: 0.7720513 (1006)	total: 1m 46s	remaining: 4m 7s
1400:	learn: 0.8759739	test: 0.7719193	best: 0.7723139 (1300)	total: 2m 2s	remaining: 3m 47s
bestTest = 0.772313884
bestIteration = 1300
Shrink model to first 1301 iterations.
Fold 4 AUC: 0.78547


Default metric period is 5 because AUC, PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7168332	test: 0.6971958	best: 0.6971958 (0)	total: 92.2ms	remaining: 6m 8s
200:	learn: 0.7870284	test: 0.7689945	best: 0.7689945 (200)	total: 18.8s	remaining: 5m 55s
400:	learn: 0.8052280	test: 0.7749333	best: 0.7750230 (385)	total: 35.1s	remaining: 5m 15s
600:	learn: 0.8185825	test: 0.7772018	best: 0.7773493 (591)	total: 50.7s	remaining: 4m 46s
800:	learn: 0.8325393	test: 0.7778777	best: 0.7781068 (751)	total: 1m 7s	remaining: 4m 28s
1000:	learn: 0.8448521	test: 0.7787285	best: 0.7789610 (991)	total: 1m 24s	remaining: 4m 12s
1200:	learn: 0.8548530	test: 0.7788472	best: 0.7790446 (1165)	total: 1m 40s	remaining: 3m 54s
1400:	learn: 0.8655616	test: 0.7787955	best: 0.7790446 (1165)	total: 1m 57s	remaining: 3m 37s
bestTest = 0.7790446105
bestIteration = 1165
Shrink model to first 1166 iterations.
Fold 5 AUC: 0.78421

AUC medio: 0.78095 ± 0.00349


In [11]:
print(aucs)

[0.7769572841438478, 0.7809312420422776, 0.7771989852953266, 0.7854652992249869, 0.7842141418850236]


In [15]:

# Probabilidades (idéntico conceptualmente)
y_val_proba = models[3].predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#optimal threshold based on expected loss
cost_FN = 10000   # default
cost_FP = 1000    # lost good client

thresholds = np.linspace(0, 1, 101)
losses = []

for t in thresholds:
    y_pred = (y_val_proba >= t).astype(int)

    FP = np.sum((y_val == 0) & (y_pred == 1))
    FN = np.sum((y_val == 1) & (y_pred == 0))

    loss = FP * cost_FP + FN * cost_FN
    losses.append(loss)

optimal_threshold = thresholds[np.argmin(losses)]

#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"Used Threshold: {optimal_threshold}")
print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_val, y_val_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")

Used Threshold: 0.53
FPR: 0.2570
Precision (PPV): 0.2123
AUC validación: 0.81331
              precision    recall  f1-score   support

           0       0.97      0.74      0.84    138182
           1       0.21      0.73      0.33     13101

    accuracy                           0.74    151283
   macro avg       0.59      0.74      0.58    151283
weighted avg       0.90      0.74      0.80    151283


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def    102663    35519
Real Default     3529     9572


In [18]:
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

y = df['TARGET']
X = df.drop(columns = ['TARGET', 'SK_ID_CURR'])

X.replace([np.inf, -np.inf], np.nan, inplace=True)

cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="constant", fill_value= 0)  #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.6, stratify=y ,random_state=42)

# Probabilidades (idéntico conceptualmente)
y_val_proba = models[3].predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#optimal threshold based on expected loss
cost_FN = 10000   # default
cost_FP = 1000    # lost good client

thresholds = np.linspace(0, 1, 101)
losses = []

for t in thresholds:
    y_pred = (y_val_proba >= t).astype(int)

    FP = np.sum((y_val == 0) & (y_pred == 1))
    FN = np.sum((y_val == 1) & (y_pred == 0))

    loss = FP * cost_FP + FN * cost_FN
    losses.append(loss)

optimal_threshold = thresholds[np.argmin(losses)]

#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_val, y_val_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")


FPR: 0.2570
Precision (PPV): 0.2123
AUC validación: 0.81331
              precision    recall  f1-score   support

           0       0.97      0.74      0.84    138182
           1       0.21      0.73      0.33     13101

    accuracy                           0.74    151283
   macro avg       0.59      0.74      0.58    151283
weighted avg       0.90      0.74      0.80    151283


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def    102663    35519
Real Default     3529     9572


In [16]:
joblib.dump(models[3], '../models/catboost_best_scores.pkl')

['../models/catboost_best_scores.pkl']